In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("covid_data_v2.csv")

In [3]:
df.head()

,country,ISO2,ISO,slug,start_date,end_date,lable_end_date,active,new_cases_feature,new_cases_label,stringency_actual,population,gdp_per_capita,population_density,Urban_population,UHC_service_coverage_index,basic_sanitation_services,Population_ages_0_to_14,Population_ages_15_to_64,Population_ages_65_and_over
0,Fiji,FJ,FJI,fiji,2020-01-22,2020-02-11,2020-03-02,0,0,0,19.44,889953.0,6220.046421,48.357033,56.248,64.0,95.072175,29.293682,65.082089,5.62423
1,Fiji,FJ,FJI,fiji,2020-02-11,2020-03-02,2020-03-22,0,0,2,19.44,889953.0,6220.046421,48.357033,56.248,64.0,95.072175,29.293682,65.082089,5.62423
2,Fiji,FJ,FJI,fiji,2020-03-02,2020-03-22,2020-04-11,2,2,16,79.63,889953.0,6220.046421,48.357033,56.248,64.0,95.072175,29.293682,65.082089,5.62423
3,Fiji,FJ,FJI,fiji,2020-03-22,2020-04-11,2020-05-01,16,14,16,88.89,889953.0,6220.046421,48.357033,56.248,64.0,95.072175,29.293682,65.082089,5.62423
4,Fiji,FJ,FJI,fiji,2020-04-11,2020-05-01,2020-05-21,6,2,2,88.89,889953.0,6220.046421,48.357033,56.248,64.0,95.072175,29.293682,65.082089,5.62423


In [4]:
X = df.drop(["new_cases_label","country","ISO2","slug","start_date","end_date","lable_end_date","ISO" ]  , 1)
y = df["new_cases_label"]

In [5]:
y = y.values.reshape(-1,1)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [37]:
X_train.head()

,active,new_cases_feature,stringency_actual,population,gdp_per_capita,population_density,Urban_population,UHC_service_coverage_index,basic_sanitation_services,Population_ages_0_to_14,Population_ages_15_to_64,Population_ages_65_and_over
5128,993,1117,53.70,361313.0,66944.825508,3.518414,93.813,84.0,98.782168,19.647171,65.160708,15.192121
7153,0,0,19.44,6453553.0,4187.250031,309.881467,72.023,76.0,87.433567,26.858445,64.671972,8.469583
4508,1003,1137,41.20,8574832.0,81993.727149,215.465350,73.797,83.0,99.890374,14.940676,66.221328,18.837996
7415,244839,13043,69.91,66834405.0,42300.267126,274.708982,83.398,87.0,99.110295,17.695701,63.795604,18.508696
1010,610,629,89.35,18513930.0,9731.145207,6.769826,57.428,76.0,97.873702,28.881250,63.466406,7.652344


In [7]:
# Scale your data
from sklearn.preprocessing import StandardScaler
X_minmax = StandardScaler().fit(X_train)
y_minmax = StandardScaler().fit(y_train)


X_train_minmax = X_minmax.transform(X_train)
X_test_minmax = X_minmax.transform(X_test)


y_train_minmax = y_minmax.transform(y_train)
y_test_minmax = y_minmax.transform(y_test)


In [8]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(random_state=0)

regr.fit(X_train_minmax, y_train_minmax.ravel())

score = regr.score(X_train_minmax, y_train_minmax)
print(f"R2 Score: {score}")


R2 Score: 0.9977983564911012


In [9]:
score = regr.score(X_test_minmax, y_test_minmax)
print(f"R2 Score: {score}")

R2 Score: 0.9881056313964731


In [10]:
feature_importances = pd.DataFrame({"feature_name" : list(X.columns), "feature_importances": list(regr.feature_importances_)})
feature_importances["feature_importances"] = feature_importances["feature_importances"]*100
feature_importances

,feature_name,feature_importances
0,active,0.760116
1,new_cases_feature,97.435884
2,stringency_actual,0.916139
3,population,0.240605
4,gdp_per_capita,0.146984
5,population_density,0.088295
6,Urban_population,0.154009
7,UHC_service_coverage_index,0.041500
8,basic_sanitation_services,0.031790
9,Population_ages_0_to_14,0.046782


In [11]:
import joblib
filename = 'RandomForestRegressor.sav'
joblib.dump(regr, filename)

['RandomForestRegressor.sav']

In [31]:
data_for_prediction = pd.read_csv("data_for_predection.csv")

In [35]:
X_values = data_for_prediction.drop(["country","ISO2","slug","start_date","end_date","ISO" ]  , 1)

In [36]:
X_values.head()

,active,new_cases_feature,stringency_actual,population,gdp_per_capita,population_density,Urban_population,UHC_service_coverage_index,basic_sanitation_services,Population_ages_0_to_14,Population_ages_15_to_64,Population_ages_65_and_over
0,8513,6573,37.04,4974986,15474.032475,15.604145,85.443,69,99.999999,22.418395,75.133015,2.448590
1,512665,910691,61.57,1366417754,2104.145900,454.938073,34.472,55,59.543448,26.618265,67.003811,6.377924
2,8812,14073,40.74,9053300,43641.397736,410.480591,92.501,82,100.000001,27.885572,59.905754,12.208674
3,123,137,43.52,69625582,7808.192916,135.897207,50.692,80,98.750735,16.823536,70.770213,12.406250
4,1277,151,43.52,11263077,754.587918,403.598549,56.192,49,34.704454,32.858587,62.077743,5.063670


In [38]:
X_values_scaled = X_minmax.transform(X_values)

In [39]:
X_values_scaled.shape

(149, 12)

In [40]:
prediction_scaled = regr.predict(X_values_scaled)

In [41]:
len(prediction_scaled)

149

In [42]:
prediction = y_minmax.inverse_transform(prediction_scaled)

In [43]:
len(prediction)

149

In [45]:
data_for_prediction["predicted"] = prediction

In [46]:
data_for_prediction.to_csv("predicted_values.csv", index = False)